In [17]:
import matplotlib
matplotlib.use('Agg')  # noqa
import matplotlib.pyplot as plt
import numpy as np

In [13]:
def forward_euler(y0, delta_t, dydt):
    y_new = y0 + delta_t * dydt
    return y_new

In [14]:
def generate_t_list(t0, dt, tmax):
    t_list = []
    # If 0<dt<1, can't use range function
    if (dt > 0 and dt < 1):
        count = t0
        while (count < tmax):
            t_list.append(count)
            count += dt
    else:  # Get list of times using range
        t_list = list(range(t0, tmax, dt))
    return t_list

In [18]:
def simulate_malaria_aging(SH_0_v, IH_0_v, EH_0_v, RH_0_v, SM_0, EM_0, IM_0, mu_H, mu_M, beta_H_v, beta_M, gamma_M, nu, sigma_v, gamma_v, delta_v, psi_v, zeta, dt, tmax):
    '''
    Numerical simulation of a malaria model with aging
    - Uniform birth/death
    - Adult and juvenile humans, one kind of mosquito
    - Calculated using absolute numbers, not proportions
    
    Humans are simulated using an SEIR model
    Mosquitoes are simulated using an SI model
    Time is in units of days
    Human paramaters are parsed into the function in a list with juvenile parameters in index 0
    
    Variables:
    mu_H = Natural human death rate
    mu_M = Natural mosquito death rate
    beta_HJ = Juvenile human exposure rate (mosquito biting rate * chance of infection per bite)
    beta_HA = Adult human exposure rate
    beta_M = Mosquito infection rate (mosquito biting rate * chance of infection after bite)
    nu = Tuning factor to scale probability that exposed individuals can pass infection to mosquito
    sigma_J = Rate of development of clinical symptoms in juvenile humans after exposure
    sigma_A = Rate of development of clinical symptoms in adult humans after exposure
    gamma_J = Juvenile human recovery rate
    gamma_A = Adult human recovery rate
    delta_J = Infection induced mortality rate in juvenile humans
    delta_A = Infection induced mortality rate in adult humans
    psi_J = Juvenile human loss of immunity rate (return to susceptible)
    psi_A = Adult human loss of immunity rate (return to susceptible)
    zeta = Aging rate for humans
    '''
    NH = sum(RH_0_v) + sum(SH_0_v) + sum(IH_0_v) + sum(EH_0_v)
    NM = SM_0 + EM_0 + IM_0
    
    # Unpack variables
    beta_HJ = beta_H_v[0]
    beta_HA = beta_H_v[1]
    sigma_J = sigma_v[0]
    sigma_A = sigma_v[1]
    gamma_J = gamma_v[0]
    gamma_A = gamma_v[1]
    delta_J = delta_v[0]
    delta_A = delta_v[1]
    psi_J = psi_v[0]
    psi_A = psi_v[1]
    
    # Storage of values over time
    SHJ = [SH_0_v[0]]
    EHJ = [EH_0_v[0]]
    IHJ = [IH_0_v[0]]
    RHJ = [RH_0_v[0]]
    
    SHA = [SH_0_v[1]]
    EHA = [EH_0_v[1]]
    IHA = [IH_0_v[1]]
    RHA = [RH_0_v[1]]
    
    SM = [SM_0]
    EM = [EM_0]
    IM = [IM_0]
    
    # Set initial values
    curr_SHJ = SH_0_v[0]
    curr_EHJ = EH_0_v[0]
    curr_IHJ = IH_0_v[0]
    curr_RHJ = RH_0_v[0]
    
    curr_SHA = SH_0_v[1]
    curr_EHA = EH_0_v[1]
    curr_IHA = IH_0_v[1]
    curr_RHA = RH_0_v[1]
    
    curr_SM = SM_0
    curr_EM = EM_0
    curr_IM = IM_0
    
    # If 0<dt<1, can't use range function
    t_list = generate_t_list(t0=0, dt=dt, tmax=tmax)

    for t in t_list:
        # dSHJ = birth + unrecovery rate - aging rate - exposure rate - death rate
        dSHJ = (mu_H * NH + delta_J * curr_IHJ + delta_A * curr_IHA) + (psi_J * curr_RHJ) - (zeta * curr_SHJ) - (beta_HJ * curr_SHJ * curr_IM / NM) - (mu_H * curr_SHJ)
        # dSHA = aging rate + unrecovery rate - exposure rate - death rate
        dSHA = (zeta * curr_SHJ) + (psi_A * curr_RHA) - (beta_HA * curr_SHA * curr_IM / NM) - (mu_H * curr_SHA)
        
        # dEHJ = Exposure rate - aging rate - infection rate - death rate
        dEHJ = (beta_HJ * curr_SHJ * curr_IM / NM) - ((zeta + sigma_J + mu_H) * curr_EHJ)
        # dEHA = Exposure rate + aging rate - infection rate - death rate
        dEHA = (beta_HA * curr_SHA * curr_IM / NM) + (zeta * curr_EHJ) - ((sigma_A + mu_H) * curr_EHA)
        
        # dIHJ = Infection rate - aging rate - recovery rate - natural death rate - disease associated mortality
        dIHJ = (sigma_J * curr_EHJ) - ((zeta + gamma_J + mu_H + delta_J) * curr_IHJ)
        # dIHA = Infection rate + aging rate - recovery rate - natural death rate - disease associated mortality
        dIHA = (sigma_A * curr_EHA) + (zeta * curr_IHJ) - ((gamma_A + mu_H + delta_A) * curr_IHA)
        
        # dRHJ = Recovery rate - aging rate - unrecovery rate - death rate
        dRHJ = (gamma_J * curr_IHJ) - ((zeta + psi_J + mu_H) * curr_RHJ)
        # dRHA = Recovery rate + aging rate - unrecovery rate - death rate
        dRHA = (gamma_A * curr_IHA) + (zeta * curr_RHJ) - ((psi_A + mu_H) * curr_RHA)
        
        # dSM = Birth - exposure rate - death rate
        dSM = (mu_M * NM) - ((beta_M * (nu * (curr_EHJ + curr_EHA) + curr_IHJ + curr_IHA) / NH) * curr_SM) - (mu_M * curr_SM)
        
        # dEM = exposure rate - infection rate - death rate
        dEM = ((beta_M * (nu * (curr_EHJ + curr_EHA) + curr_IHJ + curr_IHA) / NH) * curr_SM) - (gamma_M * curr_EM) - (mu_M * curr_EM)
        
        # dIM = Infection rate - death rate
        dIM = (gamma_M * curr_EM) - (mu_M * curr_IM)

        curr_SHJ = forward_euler(curr_SHJ, dt, dSHJ)
        curr_EHJ = forward_euler(curr_EHJ, dt, dEHJ)
        curr_IHJ = forward_euler(curr_IHJ, dt, dIHJ)
        curr_RHJ = forward_euler(curr_RHJ, dt, dRHJ)
        
        curr_SHA = forward_euler(curr_SHA, dt, dSHA)
        curr_EHA = forward_euler(curr_EHA, dt, dEHA)
        curr_IHA = forward_euler(curr_IHA, dt, dIHA)
        curr_RHA = forward_euler(curr_RHA, dt, dRHA)
        
        curr_SM = forward_euler(curr_SM, dt, dSM)
        curr_EM = forward_euler(curr_EM, dt, dEM)
        curr_IM = forward_euler(curr_IM, dt, dIM)

        SHJ.append(curr_SHJ)
        EHJ.append(curr_EHJ)
        IHJ.append(curr_IHJ)
        RHJ.append(curr_RHJ)
        
        SHA.append(curr_SHA)
        EHA.append(curr_EHA)
        IHA.append(curr_IHA)
        RHA.append(curr_RHA)
        
        SM.append(curr_SM)
        EM.append(curr_EM)
        IM.append(curr_IM)
    
    # Complete list of times, return results
    t_list.append(t_list[-1] + dt)
    
    # Create a single results array to return
    results_transpos = np.array([t_list, SHJ, EHJ, IHJ, RHJ, SHA, EHA, IHA, RHA, SM, EM, IM])
    results = results_transpos.T

    return results

In [78]:
SH_0_v = [2500000, 2500000]
EH_0_v = [0, 0]
IH_0_v = [0, 0]
RH_0_v = [0, 0]
SM_0 = 195000
EM_0 = 0
IM_0 = 5000
mu_H = 0.00004
mu_M = 0.05
beta_H_v = [0.0905, 0.0905]
beta_M = 0.417
gamma_M = 0.2  # Five days (guess right now)
nu = 0.5
sigma_v = [0.1033, 0.0833]
gamma_v = [0.0014, 0.0035]
delta_v = [0.0003454, 0.0000174]
psi_v = [0.027, 0.027]
zeta = 0.00000986
dt = 1  # Time in days
tmax = 550  # 5 years

results = simulate_malaria_aging(SH_0_v, IH_0_v, EH_0_v, RH_0_v, SM_0, EM_0, IM_0, mu_H, mu_M, beta_H_v, beta_M, gamma_M, nu, sigma_v, gamma_v, delta_v, psi_v, zeta, dt, tmax)

In [80]:
# Unpack results and plot
t_list = results[:, 0]
SHJ = results[:, 1]
EHJ = results[:, 2]
IHJ = results[:, 3]
RHJ = results[:, 4]
SHA = results[:, 5]
EHA = results[:, 6]
IHA = results[:, 7]
RHA = results[:, 8]
SM = results[:, 9]
EM = results[:, 10]
IM = results[:, 11]

In [87]:
fig, ax = plt.subplots()
ax.plot(t_list,SHJ,label='S Juvenile',color='orange',linestyle='dashed')
ax.plot(t_list,EHJ,label='E Juvenile',color='green',linestyle='dashed')
ax.plot(t_list,IHJ,label='I Juvenile',color='red',linestyle='dashed')
ax.plot(t_list,RHJ,label='R Juvenile',color='blue',linestyle='dashed')
ax.plot(t_list,SHA,label='S Adult',color='orange',linestyle='solid')
ax.plot(t_list,EHA,label='E Adult',color='green',linestyle='solid')
ax.plot(t_list,IHA,label='I Adult',color='red',linestyle='solid')
ax.plot(t_list,RHA,label='R Adult',color='blue',linestyle='solid')
ax.set_xlabel('Time (Days)')
ax.set_ylabel('Population')
ax.set_title('Human Population vs Time')
ax.legend(loc='center right')
plt.savefig('sma_humans.png',dpi=500)
plt.close()

In [88]:
fig, ax = plt.subplots()
ax.plot(t_list,SM,label='S Mosquito',color='orange')
ax.plot(t_list,EM,label='E Mosquito',color='green')
ax.plot(t_list,IM,label='I Mosquito',color='red')
ax.set_xlabel('Time (Days)')
ax.set_ylabel('Mosquito Population')
ax.set_title('Mosquito Population vs Time')
ax.legend(loc='center right')
plt.savefig('sma_mosquito.png',dpi=500)
plt.close()